In [1]:
import os
import subprocess
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

In [2]:
def create_blast_db(genome_file, db_name):
    cmd = ['makeblastdb', '-in', genome_file, '-dbtype', 'nucl', '-out', db_name]
    subprocess.run(cmd, check=True)
    print(f"BLAST database {db_name} created.")

In [3]:
def run_blast(query_file, db_name, result_file):
    cmd = ['blastn', '-query', query_file, '-db', db_name, '-out', result_file, '-outfmt', '6']
    subprocess.run(cmd, check=True)
    print(f"BLAST search completed. Results saved to {result_file}.")

In [4]:
def extract_matched_sequences(blast_result_file, genome_file, output_file):
    matches = []
    with open(blast_result_file, 'r') as blast_file:
        for line in blast_file:
            columns = line.strip().split('\t')
            query_id, subject_id, identity, alignment_length, mismatches, gap_opens, q_start, q_end, s_start, s_end, evalue, bit_score = columns
            matches.append((subject_id, int(s_start), int(s_end)))

    genome_records = SeqIO.to_dict(SeqIO.parse(genome_file, "fasta"))

    extracted_sequences = []
    for subject_id, start, end in matches:
        if start < end:
            sequence = genome_records[subject_id].seq[start-1:end]  # Convert to 0-based index
        else:
            sequence = genome_records[subject_id].seq[end-1:start].reverse_complement()
        
        record = SeqRecord(sequence, id=f"{subject_id}_{start}_{end}", description="")
        extracted_sequences.append(record)
    
    SeqIO.write(extracted_sequences, output_file, "fasta")
    print(f"Extracted sequences saved to {output_file}")

In [5]:
def find_genome_file(gca_directory):
    for file_name in os.listdir(gca_directory):
        if file_name.endswith(".fna"):
            return os.path.join(gca_directory, file_name)
    raise FileNotFoundError(f"No genome file found in directory {gca_directory}")

In [6]:
def process_directory(query_ITS, query_benA, gca_directory, gca):
    genome_file = find_genome_file(gca_directory)
    db_name = os.path.join(gca_directory, "db")
    result_ITS = os.path.join(gca_directory, "results_ITS.txt")
    result_benA = os.path.join(gca_directory, "results_benA.txt")
    output_ITS = os.path.join(gca_directory, "sequences_ITS.fasta")
    output_benA = os.path.join(gca_directory, "sequences_benA.fasta")

    create_blast_db(genome_file, db_name)
    
    run_blast(query_ITS, db_name, result_ITS)
    run_blast(query_benA, db_name, result_benA)
    
    extract_matched_sequences(result_ITS, genome_file, output_ITS)
    extract_matched_sequences(result_benA, genome_file, output_benA)

In [37]:
base_dir = "data/BLAST/"
query_ITS = base_dir + "query/" + "KC936269.1_Aspergillus_fumigatus_ITS.fasta"
query_benA = base_dir + "query/" + "KJ831653.1_Aspergillus_fumigatus_benA.fasta"
gca = "GCA_040142845.1"

gca_directory = os.path.join(base_dir, gca)
print(f"Processing directory: {gca_directory}")
process_directory(query_ITS, query_benA, gca_directory, gca)

Processing directory: data/BLAST/GCA_040142845.1


Building a new DB, current time: 10/18/2024 11:22:27
New DB name:   /disk1/01.Myeongkyu/07.Fungi_elaboration_4mer/data/BLAST/GCA_040142845.1/db
New DB title:  data/BLAST/GCA_040142845.1/GCA_040142845.1_UCR_Afum_ATCC42202_1.0_genomic.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 15 sequences in 0.0674059 seconds.


BLAST database data/BLAST/GCA_040142845.1/db created.
BLAST search completed. Results saved to data/BLAST/GCA_040142845.1/results_ITS.txt.
BLAST search completed. Results saved to data/BLAST/GCA_040142845.1/results_benA.txt.
Extracted sequences saved to data/BLAST/GCA_040142845.1/sequences_ITS.fasta
Extracted sequences saved to data/BLAST/GCA_040142845.1/sequences_benA.fasta
